In [2]:
#importing pyspark
import pyspark

In [3]:
#importing sparksession
from pyspark.sql import SparkSession

In [4]:
#creating a sparksession object and providing appName
spark = SparkSession.builder.appName("business_case_sql").getOrCreate()

In [5]:
#To create dataframe form external datasets
AirlineDF = spark.read.option("header","true").csv("airlines1.csv")

To register Dataframe as a table 

In [6]:
AirlineDF.registerTempTable("AirlineTable")

C:\Users\User\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
#How to read the register table
spark.sql("SELECT * FROM AirlineTable").show(5)

+---+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+--------------------+---------+-------------+--------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----

In [8]:
#count number of rows
spark.sql("SELECT count(*) FROM AirlineTable").show()

+--------+
|count(1)|
+--------+
|   50001|
+--------+



In [9]:
#How to see all the columns with the datatype
spark.sql("describe AirlineTable").show(50)

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                 _c0|   string|   null|
|                Year|   string|   null|
|             Quarter|   string|   null|
|               Month|   string|   null|
|          DayofMonth|   string|   null|
|           DayOfWeek|   string|   null|
|          FlightDate|   string|   null|
|   Reporting_Airline|   string|   null|
|DOT_ID_Reporting_...|   string|   null|
|IATA_CODE_Reporti...|   string|   null|
|         Tail_Number|   string|   null|
|Flight_Number_Rep...|   string|   null|
|     OriginAirportID|   string|   null|
|  OriginAirportSeqID|   string|   null|
|  OriginCityMarketID|   string|   null|
|              Origin|   string|   null|
|      OriginCityName|   string|   null|
|         OriginState|   string|   null|
|     OriginStateFips|   string|   null|
|     OriginStateName|   string|   null|
|           OriginWac|   string|   null|
|       DestAirp

In [10]:
#Select some columns form table 
spark.sql("SELECT origin, destcityname FROM AirlineTable").show(5)

+------+--------------------+
|origin|        destcityname|
+------+--------------------+
|   MSP|  Salt Lake City, UT|
|   MKE|         Orlando, FL|
|   GJT|Dallas/Fort Worth...|
|   LAX|         Detroit, MI|
|   EWR|       Charlotte, NC|
+------+--------------------+
only showing top 5 rows



In [11]:
#Average departure delay
spark.sql("SELECT avg(DepDelayMinutes) as Avg_Delay_Time FROM AirlineTable").show()

+------------------+
|    Avg_Delay_Time|
+------------------+
|10.851455322613475|
+------------------+



In [12]:
#Calculate overall flights delay 
spark.sql("SELECT Flight_Number_Reporting_Airline, DepDelay+ArrDelay as Total_delay FROM AirlineTable").show()

+-------------------------------+-----------+
|Flight_Number_Reporting_Airline|Total_delay|
+-------------------------------+-----------+
|                            675|       42.0|
|                            671|       -2.0|
|                           3297|       11.0|
|                           1806|      -20.0|
|                            465|       83.0|
|                           1198|       11.0|
|                           1431|       -2.0|
|                           3459|      435.0|
|                           7233|       12.0|
|                           5932|       45.0|
|                           2135|        8.0|
|                           1463|       -5.0|
|                            830|      -13.0|
|                           6474|      -23.0|
|                            674|      -21.0|
|                            362|       10.0|
|                           1678|      -18.0|
|                            413|      -35.0|
|                            210| 

In [13]:
#Count total number of flights by origin airport and toal avg and maximum DepDelay time in each airport
spark.sql("SELECT origin, COUNT(*), avg(DepDelayMinutes), max(DepDelayMinutes) FROM AirlineTable GROUP BY origin ").show()

+------+--------+--------------------+--------------------+
|origin|count(1)|avg(DepDelayMinutes)|max(DepDelayMinutes)|
+------+--------+--------------------+--------------------+
|   ABE|      49|   6.659574468085107|                44.0|
|   ABI|      11|  0.5555555555555556|                 4.0|
|   ABQ|     290|   9.041958041958042|                 9.0|
|   ABR|       3|               284.0|               852.0|
|   ABY|       3|                20.0|                30.0|
|   ACK|       1|                 0.0|                 0.0|
|   ACT|      16|              3.1875|                33.0|
|   ACV|      14|   7.071428571428571|                 7.0|
|   ACY|       5|                 7.6|                38.0|
|   ADK|       2|                 6.5|                13.0|
|   ADQ|       4|   4.666666666666667|                14.0|
|   AEX|      11|   8.090909090909092|                34.0|
|   AGS|      27|  23.814814814814813|                64.0|
|   AKN|       2|                 9.5|  

In [14]:
#calculate origin state wise cancelled flights
spark.sql("SELECT OriginState, count(Cancelled) FROM AirlineTable WHERE Cancelled = 1 GROUP BY originState ").show(5)

+-----------+----------------+
|OriginState|count(Cancelled)|
+-----------+----------------+
|         SC|               8|
|         AZ|              12|
|         MN|              26|
|         NJ|              27|
|         OR|               5|
+-----------+----------------+
only showing top 5 rows



In [15]:
#pass condition one two columns
spark.sql("SELECT origin, COUNT(*) FROM AirlineTable WHERE year = 2010 and month = 6 GROUP BY origin ORDER BY COUNT(*) DESC").show()

+------+--------+
|origin|count(1)|
+------+--------+
|   ATL|       8|
|   ORD|       7|
|   DFW|       6|
|   MCO|       6|
|   PHX|       6|
|   LAX|       5|
|   IAH|       5|
|   CLE|       4|
|   SFO|       3|
|   LAS|       3|
|   MEM|       3|
|   BOS|       3|
|   CMH|       3|
|   IAD|       3|
|   CLT|       3|
|   TPA|       3|
|   MIA|       3|
|   EWR|       2|
|   DEN|       2|
|   SMF|       2|
+------+--------+
only showing top 20 rows



In [16]:
#pass more than two onditions on different columns 
spark.sql("SELECT DOT_ID_Reporting_Airline FROM AirlineTable WHERE year = 2015 and month = 12 AND origin = 'JFK'").show()

+------------------------+
|DOT_ID_Reporting_Airline|
+------------------------+
|                   20409|
|                   19805|
+------------------------+



In [17]:
#Rank and number of diverted flight by origin
spark.sql("SELECT origin, sum(diverted), RANK() OVER(ORDER BY sum(diverted) DESC) AS RANKING FROM AirlineTable GROUP BY origin").show()

+------+-------------+-------+
|origin|sum(diverted)|RANKING|
+------+-------------+-------+
|   DEN|          6.0|      1|
|   ATL|          6.0|      1|
|   ORD|          6.0|      1|
|   MSP|          6.0|      1|
|   DFW|          5.0|      5|
|   FLL|          5.0|      5|
|   CVG|          4.0|      7|
|   CLT|          4.0|      7|
|   LAX|          4.0|      7|
|   MCO|          4.0|      7|
|   CMH|          3.0|     11|
|   SEA|          3.0|     11|
|   PDX|          3.0|     11|
|   BWI|          3.0|     11|
|   STL|          3.0|     11|
|   PIT|          3.0|     11|
|   DCA|          2.0|     17|
|   PSG|          2.0|     17|
|   BNA|          2.0|     17|
|   IAH|          2.0|     17|
+------+-------------+-------+
only showing top 20 rows



<h1> Case Study 1 </h1>

To calculate year wise on time flight performance

In [18]:
df1 = spark.sql("SELECT year, COUNT(*) AS TOTAL_FLIGHT, COUNT(arrdelay==0) AS TOTAL_ON_TIME, sum(cancelled) AS TOTAL_CANCELLED, sum(diverted) AS TOTAL_DIVERTED FROM AirlineTable WHERE GROUP BY YEAR; ")

In [19]:
df1.registerTempTable("AirlineTable1")

In [20]:
#performance in percentage
spark.sql("SELECT * FROM AirlineTable1").show()

+----+------------+-------------+---------------+--------------+
|year|TOTAL_FLIGHT|TOTAL_ON_TIME|TOTAL_CANCELLED|TOTAL_DIVERTED|
+----+------------+-------------+---------------+--------------+
|1987|         339|          336|            3.0|           0.0|
|2016|        1502|         1484|           17.0|           1.0|
|2020|         462|          438|           24.0|           0.0|
|2012|        1519|         1497|           20.0|           2.0|
|1988|        1310|         1293|           15.0|           2.0|
|2019|        1900|         1862|           34.0|           4.0|
|2017|        1449|         1421|           22.0|           6.0|
|2014|        1513|         1476|           31.0|           6.0|
|2013|        1690|         1663|           22.0|           5.0|
|2005|        1812|         1776|           33.0|           3.0|
|2000|        1477|         1432|           42.0|           3.0|
|2002|        1371|         1360|           11.0|           0.0|
|2009|        1604|      

In [21]:
#performance in percentage
spark.sql("SELECT year, TOTAL_FLIGHT, TOTAL_ON_TIME, TOTAL_ON_TIME*100/TOTAL_FLIGHT AS ON_TIME_PER, TOTAL_CANCELLED,TOTAL_CANCELLED*100/TOTAL_FLIGHT AS CANCELLED_PER, TOTAL_DIVERTED, TOTAL_DIVERTED*100/TOTAL_FLIGHT AS DIVERTED_PER FROM AirlineTable1;").show()

+----+------------+-------------+-----------------+---------------+------------------+--------------+-------------------+
|year|TOTAL_FLIGHT|TOTAL_ON_TIME|      ON_TIME_PER|TOTAL_CANCELLED|     CANCELLED_PER|TOTAL_DIVERTED|       DIVERTED_PER|
+----+------------+-------------+-----------------+---------------+------------------+--------------+-------------------+
|1987|         339|          336|99.11504424778761|            3.0|0.8849557522123894|           0.0|                0.0|
|2016|        1502|         1484|98.80159786950732|           17.0|1.1318242343541944|           1.0|0.06657789613848203|
|2020|         462|          438| 94.8051948051948|           24.0| 5.194805194805195|           0.0|                0.0|
|2012|        1519|         1497|98.55167873601053|           20.0| 1.316655694535879|           2.0| 0.1316655694535879|
|1988|        1310|         1293|98.70229007633588|           15.0|1.1450381679389312|           2.0|0.15267175572519084|
|2019|        1900|     

In [24]:
#To calculate origincityname wise on time flight performance
df2 = spark.sql("SELECT origincityname, COUNT(*) AS TOTAL_FLIGHT, COUNT(arrdelay == 0) AS TOTAL_ON_TIME, sum(cancelled) AS TOTAL_DIVERTED FROM AirlineTable where GROUP BY origin, origincityname").show();

+--------------------+------------+-------------+--------------+
|      origincityname|TOTAL_FLIGHT|TOTAL_ON_TIME|TOTAL_DIVERTED|
+--------------------+------------+-------------+--------------+
|Colorado Springs, CO|         100|          100|           0.0|
|      Louisville, KY|         175|          170|           4.0|
|College Station/B...|          10|           10|           0.0|
|          Pierre, SD|           1|            1|           0.0|
|         Madison, WI|          81|           80|           1.0|
|      Evansville, IN|          18|           18|           0.0|
|Champaign/Urbana, IL|          10|           10|           0.0|
|        Columbus, MS|           2|            2|           0.0|
|        Richmond, VA|         146|          141|           5.0|
|   Christiansted, VI|          11|           11|           0.0|
|        Florence, SC|           4|            4|           0.0|
|      Washington, DC|         704|          683|          19.0|
|       El Centro, CA|   

In [28]:
#speed of flights in miles/hour

df3 = spark.sql("SELECT AirTime/60 As AirTime_Hour, Distance FROM AirlineTable")

In [29]:
df3.registerTempTable("AirlineTable3")

In [30]:
spark.sql("SELECT AirTime_Hour, Distance, Distance/AirTime_Hour as nmph FROM AirlineTable3").show()

+------------------+--------+------------------+
|      AirTime_Hour|Distance|              nmph|
+------------------+--------+------------------+
|              2.55|   991.0| 388.6274509803922|
|              2.35|  1066.0| 453.6170212765957|
|1.7166666666666666|   773.0|450.29126213592235|
|3.6666666666666665|  1979.0| 539.7272727272727|
|1.3333333333333333|   529.0|            396.75|
|0.4666666666666667|   190.0| 407.1428571428571|
|1.5666666666666667|   563.0|359.36170212765956|
|0.5833333333333334|   192.0| 329.1428571428571|
|0.9833333333333333|   316.0|321.35593220338984|
|               1.9|   793.0| 417.3684210526316|
|              null|   109.0|              null|
|1.2833333333333334|   562.0| 437.9220779220779|
|              null|  1045.0|              null|
|1.5833333333333333|   677.0| 427.5789473684211|
|              1.65|   733.0|444.24242424242425|
|              null|   278.0|              null|
|               0.4|    98.0|             245.0|
|               1.7|

In [35]:
df4 = spark.sql("SELECT AirTime_Hour, Distance, Distance/AirTime_Hour as nmph FROM AirlineTable3")
df4.registerTempTable("AirlineTable4")

In [36]:
spark.sql("SELECT max(nmph) FROM AirlineTable4").show()

+---------+
|max(nmph)|
+---------+
|  17520.0|
+---------+

